# Assignment 2: Mining Itemsets (Part II)

## Finding Frequent Itemsets with Apriori

In Part I of this assignment, the summary statistics gave us a brief view of what the data look like. Now it is time for the real business - let's use the *Apriori* algorithm to find the frequent itemsets. 
First, let's import the packages and dependencies that will be used in this part of the assignment.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

from mlxtend.frequent_patterns import apriori

import time
from sklearn.preprocessing import MultiLabelBinarizer

from mlxtend.frequent_patterns import association_rules

In [ ]:
# Ignore warnings generated by mlxtend

**<span style="color:red">NOTE: These are all the imports we need to make for this assignment. You should not make other imports in your submitted notebook. You will receive 0 points for the exercises if your solution includes additional imports.</span>**

Before you practice Apriori on the Instacart dataset, let's use another dataset as an example to get familiar with the algorithm. For this assignment, **we sampled 10,000 Tweets with two or more food/drink emojis** (yes, those colorful tasty ideograms). You will represent this dataset as a collection of itemsets and practice what we learned in class.

In [ ]:
# Load the tweets data from files.
tweets_df = pd.read_csv("assets/food_drink_emoji_tweets.txt", sep="\t",header=None)
tweets_df.columns = ['text']

# Only the emojis below are considered in this assignment.
emoji_list = "🍇🍈🍉🍊🍋🍌🍍🥭🍎🍏🍐🍑🍒🍓🥝🍅🥥🥑🍆🥔🥕🌽🌶🥒🥬🥦🍄🥜🌰🍞🥐🥖🥨🥯🥞🧀🍖🍗🥩🥓🍔🍟🍕🌭🥪🌮🌯🥙🥚🍳🥘🍲🥣🥗🍿🧂🥫🍱🍘🍙🍚🍛🍜🍝🍠🍢🍣🍤🍥🥮🍡🥟🥠🥡🦀🦞🦐🦑🍦🍧🍨🍩🍪🎂🍰🧁🥧🍫🍬🍭🍮🍯🍼🥛☕🍵🍶🍾🍷🍸🍹🍺🍻🥂🥃"
emoji_set = set(emoji_list)

# Get a list of emojis used in each tweet.
tweets_df['emojis'] = tweets_df.text.apply(lambda text:np.unique([chr for chr in text if chr in emoji_set]))

# Create a matrix for Apriori algorithms.
mlb = MultiLabelBinarizer()
emoji_data = mlb.fit_transform(tweets_df.emojis).astype(bool)
emoji_matrix = pd.DataFrame(data=emoji_data, index=tweets_df.index, columns=mlb.classes_)

We can call the Apriori API now and specify the minimal support we want. You may learn more about this API from its [documentation](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/).

In [ ]:
# Apply the Apriori algorithms and find the most frequent items.
freq_emoji_itemsets = apriori(emoji_matrix, min_support=0.005, use_colnames=True)
freq_emoji_itemsets.sort_values("support", ascending=False).head()

With the above command, we preserve all itemsets with a min support of 0.005 (half percent of the shopping baskets). We can now use the following command to extract the frequent itemsets with length 2 and beyond.

In [ ]:
# Apply the Apriori algorithms and find the most frequent 2-itemsets.
freq_emoji_itemsets[freq_emoji_itemsets["itemsets"].apply(
    lambda x: len(x) > 1)].sort_values("support", ascending=False).head()


**Now, it is time to apply the Apriori algorithm to the Instacart dataset.**

Since we have already shown you how to transform the Instacart dataset into itemsets, we concatenate the data preprocessing code into one block. Please run the following code block to load and preprocess the data. Notice that we limit the number of products in this problem to 100 by popularity so that the apriori algorithm will not run for a long time.

In [ ]:
# Load data from files.
orders = pd.read_csv("assets/orders.csv.zip", nrows=10000)
products = pd.read_csv("assets/products.csv.zip")

# Group orders by order id and merge them into a list.
order_baskets = orders.groupby("order_id")["product_id"].apply(list)

# Convert the above pandas Series to a pandas DataFrame.
order_baskets = order_baskets.to_frame(name="products_id")

# Create the name map for later reference.
product_name_map = dict(zip(products["product_id"], products["product_name"]))

# Create the matrix like Part 1.
mlb = MultiLabelBinarizer()
data = mlb.fit_transform(order_baskets["products_id"]).astype(bool)
prod_matrix = pd.DataFrame(
    data=data, index=order_baskets.index, columns=mlb.classes_
)
prod_popularity = prod_matrix.sum(axis=0)
prod_matrix = prod_matrix.astype(bool)
top_prods = prod_popularity.sort_values(ascending=False).head(100).index
prod_matrix = prod_matrix[top_prods]
prod_matrix.head()

In [ ]:
prod_matrix.shape

### Exercise 2.  (20 pts)
Complete the following `prod_frequent_itemsets` function to find all the **frequent *k*-itemsets** with a minimal support of **`min_support`** in the Instacart dataset. 

Your function should return a Pandas DataFrame object like the `freq_emoji_itemsets` object above, being the default format returned by the `apriori` function. 

Make sure that you are only returning the frequent itemsets that have the specified number of products (`k`).

Hint: 

Code in the previous blocks shows you how to obtain itemsets with a length of greater than 1. How might you change the section of the code `freq_emoji_itemsets[freq_emoji_itemsets["itemsets"].apply(lambda x: len(x) > 1)]` to instead obtain itemsets equal to (==) k instead of greater than (>) 1?

In [ ]:
def prod_frequent_itemsets(prod_matrix, min_support=0.005, k=3):
    freq_sets = ""
    # YOUR CODE HERE
    raise NotImplementedError()
    return freq_sets

If you implemented this function correctly, we can obtain all frequent 2-itemsets with a min support of 0.004 by running the following command.

In [ ]:
prod_frequent_3itemsets = prod_frequent_itemsets(prod_matrix, min_support=0.004, k=3)
prod_frequent_3itemsets

In [ ]:
prod_frequent_3itemsets = prod_frequent_itemsets(prod_matrix,
                                                 min_support=0.004,
                                                 k=3)
for row in prod_frequent_3itemsets.itertuples():
    assert row.support >= 0.004, f"[Exercise 2] The support of the itemset {row.itemsets} is below the threshold."
    assert len(row.itemsets) == 3, f"[Exercise 2] The itemset {row.itemsets} is not a 3-itemset."


Let's replace the product IDs with their names. Does the result make sense to you?

In [ ]:
prod_frequent_3itemsets.itemsets = prod_frequent_3itemsets.itemsets.apply(lambda row: [product_name_map[i] for i in row])
prod_frequent_3itemsets

## Evaluating Frequent Itemsets (Part III.)

Even though we have found all the frequent itemsets, not all of them are interesting. In Part IV of this assignment, we will practice how to evaluate the frequent itemsets.

People have developed various measurements of the interestingness of patterns. Most of them split the itemset into an antecedent item(set) and a consequent item(set), and then measure the correlation between the antecedent and the consequent. Let's try some of such measurements implemented by the `mlxtend.frequent_patterns.association_rules` API, which we have imported. For more information about the API, visit the [documentation](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/) of the `mlxtend` package.

In [ ]:
prod_frequent_itemsets_basic = apriori(prod_matrix, min_support=0.005, use_colnames=True)



interestingness_measurements = association_rules(prod_frequent_itemsets_basic, metric="lift", min_threshold=0)
interestingness_measurements.head(10)

In the returned data frame, each row examines one (antecedent -> consequent) pair. *Antecedent support* and *consequent support* measure *P*(antecedent) and *P*(consequent), while *support* measures *P*(antecedent, consequent). In fact, these three values help us characterize the $2\times2$ contingency table, as illustrated in the following table:
 
 |           |              |       |              | 
 -----------:|:------------:|-------|---------------
 |           |    X = 1     | X = 0 |   sum(row)   |
 |     Y = 1 |   `support`    |       | `cons_support` |
 |     Y = 0 |              |       |              |
 | sum(col.) | `ante_support` |       |              |

Most interestingness measurements, including the four shown in the data frame (*confidence*, *lift*, *leverage*, and *conviction*), can be derived from the three support values. For example, $$\text{confidence}=\frac{\text{support}}{\text{antecedent\_support}},$$ and $$\text{lift} =\frac{\text{confidence}}{\text{consequent\_support}}=\frac{\text{support}}{\text{antecedent\_support} * \text{consequent\_support}}$$

### Exercise 3. (15 pts)
In this exercise, we are going to implement another interestingness
measurement, the (full) mutual information which we will use when adding a 'mutual information'
column to the data frame. The measurement is defined as

$$I(X;Y)=\sum_{x\in\mathcal{X}}\sum_{y\in\mathcal{Y}} P(X=x,
Y=y)\log_2\frac{P(X=x,Y=y)}{P(X=x)P(Y=y)}.$$

(*The [mutual information](https://en.wikipedia.org/wiki/Mutual_information)
quantifies the "amount of information obtained from one random variable by
observing the other random variable.*)

Note that the logorithm requirest that the joint probability $P(X=x, Y=y) > 0$,
which does not hold for some $(x, y)$. However, since we know that when $P(X=x,
Y=y) = 0$, it would not contribute to the sum, you may assume $P(X=x,
Y=y)\log_2\frac{P(X=x,Y=y)}{P(X=x)P(P=y)} = 0$ in that case. 

$x$, $y$ are possible values of $X$ and $Y$; in the case of appearance or
absence of an item, 1 or 0. Therefore, we need to consider all possible
combinations of $x$ and $y$, that is, $(X=1, Y=1)$, $(X=1, Y=0)$, $(X=0, Y=1)$,
$(X=0, Y=0)$.

Please complete the following function that uses the three support values to
compute the mutual information and return it as a float. All the three parameters are in [0, 1], and you
can assume the validity of the input. **Use 2 as the log base.** We have
created some auxilary variables for you, each represent a joint or marginal
(let $X$ denote the antecedent item and $Y$ denote the consequent item)
probability.

In [ ]:
def mi(antecedent_support, consequent_support, support):
       
    px1 = antecedent_support
    px0 = 1 - antecedent_support
    py1 = consequent_support
    py0 = 1 - consequent_support
    
    px1y1 = support
    px1y0 = px1 - px1y1
    px0y1 = py1 - px1y1
    px0y0 = 1 - px1 - py1 + px1y1
    
    mutual_information = 0
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return mutual_information

In [ ]:
# This code block tests whether the `mi` function work as expected.
# We hide some tests, so passing all the displayed assertions does not guarantee the bonus points.

assert np.abs(mi(0.6, 0.75, 0.4) - 0.04287484674660057) < 1e-8
assert np.abs(mi(0.5, 0.5, 0.25) - 0) < 1e-8

# If you fail the following assertion, double check if your function 
# handles the scenarios in which a joint probability is zero.
assert np.abs(mi(0.5, 0.5, 0.5) - 1) < 1e-8



With the `mi` function, we can now compute the mutual information for each (antecedent -> consequent) pair and attach it to the data frame. Does the result make sense?

Mutual information is a classical measure of interestingness. We encourage you to think about the following questions (not graded):
1. What is the maximum of mutual information in this setting? How to reach it?
2. What is the minimum of mutual information in this setting? How to reach it?

What does the max/min value imply?

In [ ]:
interestingness_measurements['mi'] = \
    interestingness_measurements.apply(lambda pair: mi(pair['antecedent support'], 
                                              pair['consequent support'], 
                                              pair['support']),
                                       axis=1)
interestingness_measurements.sort_values('mi', ascending=False).head(n=5)